# Using Pandas and PixieDust to explore canned craft beer in California

After exploring some (canned) craft beer data I found [here](https://www.kaggle.com/nickhould/craft-cans), I wanted to go one step further and enrich it with geographical data and map it out since I'd never tried to do so before.  However, I was thrown off by the complexity of all the Python mapping tutorials I had come across, so I looked into [PixieDust](https://ibm-cds-labs.github.io/pixiedust/) for a simpler solution.  Not only was I able to put together a nice-looking map with a few clicks, but I realized I was able to do all the graphing (à la Matplotlib) I needed to do to learn some cool things about the dataset, including:

- Does higher IBU correlate with higher ABV?
- Which city in California brews the hoppiest/most alcoholic beer?
- Does geographical region (Northern vs Southern California) affect the type of beer a brewery chooses to brew?
- Does geographical region correspond to IBU/ABV?

With the insights above, we could plan a craft brewery tour in California (e.g. if we wanted to go to many breweries within one region, or if we wanted to try a wide variety of beers within one city).  If we were really indecisive, then we could use the Yelp API to pull brewery review data and simply map out where the best-reviewed breweries are (code is below as well).

PixieDust is still a very recent tool so there are still a few things which I'd like to be able to do, but I love how easy it is to generate visualizations and learn interesting things from my data.

In this notebook, we'll use Pandas to clean and combine the initial datasets, and [geopy](https://github.com/geopy/geopy) to add some geographical information to the data.  We'll then convert the Pandas dataframe to a Spark dataframe so we can visualize it with PixieDust.  As an optional step, we'll use the Yelp API to add brewery reviews to the data.

In [1]:
import pandas as pd
import numpy as np

# data from https://www.kaggle.com/nickhould/craft-cans (scraped from CraftCans.com in January 2017)
beers = pd.read_csv("beers.csv",encoding='utf-8')
breweries = pd.read_csv("breweries.csv",encoding='utf-8')

# remove redundant column, rename columns for clarity
beers = beers.drop('Unnamed: 0',axis=1)
breweries = breweries.rename(columns = {'Unnamed: 0': 'brewery_id', 'name': 'brewery_name'})

# merge dataframes, remove NaN values, make ABV more readable, remove leading whitespace
data = pd.merge(beers,breweries,on='brewery_id',how='inner')
data = data[np.isfinite(data['ibu'])]
data['abv'] = data['abv']*100
data['state'] = data['state'].str.slice(1,3)

# get data from California
ca = data[data['state'] == 'CA']

In [2]:
ca['brewery_name'].value_counts()

21st Amendment Brewery               17
Golden Road Brewing                  14
Anderson Valley Brewing Company      14
Modern Times Beer                     8
TailGate Beer                         7
Mike Hess Brewing Company             6
Sierra Nevada Brewing Company         6
Manzanita Brewing Company             5
Ruhstaller Beer Company               5
Black Market Brewing Company          4
Ballast Point Brewing Company         4
Fort Point Beer Company               4
Central Coast Brewing Company         4
Saint Archer Brewery                  4
Hess Brewing Company                  3
Mission Brewery                       3
Devil's Canyon Brewery                3
The Dudes' Brewing Company            3
Firestone Walker Brewing Company      3
Headlands Brewing Company             3
Mavericks Beer Company                3
Hangar 24 Craft Brewery               2
Figueroa Mountain Brewing Company     2
Butcher's Brewing                     1
Mother Earth Brew Company             1


In [3]:
ca['city'].value_counts()

San Diego              35
San Francisco          22
Boonville              14
Los Angeles            14
Chico                   6
Temecula                5
Santee                  5
Sacramento              5
San Luis Obispo         4
Torrance                3
Paso Robles             3
Half Moon Bay           3
Belmont                 3
Mill Valley             3
Redlands                2
Santa Cruz              2
Buellton                2
Claremont               1
Vista                   1
South San Francisco     1
Carlsbad                1
Name: city, dtype: int64

In [4]:
# use geopy to get coordinate data so we can use PixieDust's mapping capabilities
from geopy.geocoders import Nominatim
geolocator = Nominatim()

# make it easier for geopy to find coordinates
ca['city-state'] = ca['city']+", "+ca['state']
cities = ca['city-state'].unique()

# create dictionaries of latitudes and longitudes
lats = dict(zip(cities, pd.Series(cities).apply(geolocator.geocode).apply(lambda x: x.latitude)))
longs = dict(zip(cities, pd.Series(cities).apply(geolocator.geocode).apply(lambda x: x.longitude)))

ca['latitude'] = ca['city-state'].map(lats)
ca['longitude'] = ca['city-state'].map(longs)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-c

All we need to do now is to import PixieDust, convert the dataframe to a Spark dataframe, and then use PixieDust's `display` API to visualize the data!

In [5]:
import pixiedust

# convert Pandas dataframe to Spark dataframe for visualization with PixieDust
sqlContext = SQLContext(sc)
ca2 = sqlContext.createDataFrame(ca)

display(ca2)

(optional step: add Yelp reviews to brewery data)

In [6]:
# use the Yelp API to get reviews for each brewery in the list
# API tokens have been redacted so this step won't work unless you put your own in

import reviews

def rev(r):
    return reviews.query_api(r['brewery_name'],r['city-state'])

bs = ca[['brewery_name','city-state','latitude','longitude']]
bs = bs.drop_duplicates()
bs['review'] = bs.apply(rev,axis=1)

bs2 = sqlContext.createDataFrame(bs)
display(bs2)